# Ch02. 텍스트 전처리 (Text Preprocessing)

# v06. 정수 인코딩 (Integer Encoding)

**텍스트를 숫자로 바꾸는 기법을 적용하기 위한 첫 단계**

- 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업
- ex) 가지고 있는 텍스트에 5,000개의 단어가 있다.
  - 5,000개의 단어들 각각에 1번부터 5,000번까지 단어와 맵핑되는 **고유한 정수(인덱스)**를 부여
    - book -> 150번
    - dog -> 171번
    - love -> 192번
    - books -> 212번

**인덱스를 부여하는 방법**

- 랜덤으로 부여
- 전처리 또는 빈도수가 높은 단어들만 사용하기 위해서 단어에 대한 빈도수를 기준으로 정렬한 뒤에 부여

<br>

## 6.1 정수 인코딩 (Integer Encoding)

- 어떤 과정으로 단어에 정수 인덱스를 부여하는 지 확인

**단어의 정수를 부여하는 방법**

- 단어를 빈도수 순으로 정렬한 단어 집합(vocabulary)을 만듬
- 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여

- 단어의 빈도수가 적당하게 분포되도록 의도적으로 만든 텍스트 데이터를 가지고 실습

<br>

### 6.1.1 dictionary 사용하기

In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
text="A barber is a person. a barber is good person. a barber is huge person. \
he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. \
a barber kept his word. His barber kept his secret. \
But keeping and keeping such a huge secret to himself was driving the barber crazy. \
the barber went up a huge mountain."

<br>

#### 6.1.1.1 문장 토큰화

In [6]:
# 문장 토큰화
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


<br>

#### 6.1.1.2 정제 및 단어 토큰화

- 정제 작업을 병행하며 단어 토큰화 수행

In [ ]:
# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
  sentence = word_tokenize(i) # 단어 토큰화 수행
  result = []

  for word in sentence:
    word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄임
    if word not in stop_words: # 단어 토큰화된 결과에 대해서 불용어 제거
      if len(word) > 2: # 단어 길이가 2이하인 경우에 대해 단어 제거
        result.append(word)
        if word not in vocab:
          vocab[word] = 0
        vocab[word] += 1
  sentences.append(result)

In [10]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


<br>


#### 6.1.1.3 단어별 빈도수 확인

In [11]:
# 중복을 제거한 단어와 각 단어에 대한 빈도수
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


- 단어를 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어 있음

In [12]:
print(vocab["barber"]) # "barber" 라는 단어의 빈도수 출력

8


<br>

#### 6.1.1.4 빈도수가 높은 순서대로 정렬

In [13]:
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


<br>

#### 6.1.1.5 높은 빈도수를 가진 단어 순서대로 정수 인덱스 부여

In [ ]:
word_to_index = {}
i = 0

for (word, frequency) in vocab_sorted:
  if frequency > 1: # 정제(Cleaning) 챕터에서 언급했듯이 빈도수가 적은 단어 제외
    i = i+1
    word_to_index[word] = i  

In [16]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


<br>

#### 6.1.1.6 빈도수 상위 n개 단어 사용

- 텍스트 데이터에 있는 단어를 모두 사용하기 보다는 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많다.
- 위 단어들은 빈도수가 높은 순으로 낮은 정수가 부여되어 있음
- 빈도수 상위 n개의 단어만 사용하고 싶다.  
$\rightarrow$ `vocab`에서 정수값이 1부터 n까지인 단어들만 사용

- 상위 5개 단어만 사용

In [ ]:
vocab_size = 5

words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거

for w in words_frequency:
  del word_to_index[w] # 해당 단어에 대한 인덱스 정보 삭제

In [18]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- `word_to_index`에는 